<a href="https://colab.research.google.com/github/sharmapukar217/docker-images/blob/main/Clamphook_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
import io

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from google.colab import files

uploaded = files.upload()

Saving sagarmatha-scanned.xlsx - Worksheet.csv to sagarmatha-scanned.xlsx - Worksheet.csv


In [8]:
#url = "https://docs.google.com/spreadsheets/d/1IvMT-XVYvxL5n5Q7tEJaubRjS_ZYrBNL"
# df = pd.read_excel(url)
df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]) ,sep=",")

In [9]:
df

,File name,c1.Question001,c1.Question002,c1.Question003,c1.Question004,c1.Question005,c1.Question006,c1.Question007,c1.Question008,c1.Question009,...,c4.Question098,c4.Question099,c4.Question100,roll.r1,roll.r2,roll.r3,roll.r4,roll.r5,symbol number,Set.Question106
0,img08252023_0007-505,C,C,C,C,A,D,B,D,D,...,A,C,A,0.0,0.0,3.0,2.0,3.0,323.0,A
1,img08252023_0007-506,C,B,D,A,A,D,B,D,D,...,B,D,D,0.0,0.0,3.0,4.0,6.0,346.0,A
2,img08252023_0007-503,C,B,D,A,A,D,B,NaN,D,...,B,C,A,0.0,0.0,3.0,4.0,3.0,343.0,A
3,img08252023_0007-504,C,A,C,C,A,D,B,D,D,...,A,C,C,0.0,0.0,3.0,2.0,2.0,322.0,A
4,img08252023_0007-501,C,C,C,A,D,D,NaN,D,D,...,A,D,B,0.0,0.0,3.0,2.0,5.0,325.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,img08252023_0007-362,NaN,A,C,C,C,D,NaN,D,D,...,A,C,A,0.0,0.0,1.0,0.0,7.0,107.0,A
561,img08252023_0007-483,C,A,C,C,A,D,D,C,D,...,B,NaN,D,0.0,0.0,4.0,3.0,8.0,438.0,A
562,img08252023_0007-480,B,A,D,A,D,C,B,D,D,...,A,C,B,0.0,0.0,4.0,2.0,9.0,429.0,A
563,img08252023_0007-360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,...,NaN,NaN,NaN,0.0,0.0,1.0,0.0,1.0,101.0,A


### Upload Solution sets here

In [22]:
answer_a = files.upload()
answer_b = files.upload()

Saving set-a-acme-solution - Worksheet.csv to set-a-acme-solution - Worksheet (1).csv


Saving set-b-acme-solution - Worksheet.csv to set-b-acme-solution - Worksheet (1).csv


In [23]:
series_set_a = pd.read_csv(io.BytesIO(list(answer_a.values())[0])).squeeze("index")
series_set_b = pd.read_csv(io.BytesIO(list(answer_b.values())[0])).squeeze("index")
# series_set_b.reset_index()
# series_set_b.reset_index(drop=True)
series_set_a=series_set_a[1:]
series_set_a

c1.Question001    A
c1.Question002    B
c1.Question003    C
c1.Question004    D
c1.Question005    D
                 ..
c4.Question096    C
c4.Question097    D
c4.Question098    D
c4.Question099    A
c4.Question100    B
Name: 0, Length: 100, dtype: object

In [25]:
# df = df.dropna(axis=0,subset=['Set.Question106'])
# df = df.rename(columns={'Set.Question106': 'Set'})
questions = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79', 'Q80', 'Q81', 'Q82', 'Q83', 'Q84', 'Q85', 'Q86', 'Q87', 'Q88', 'Q89', 'Q90', 'Q91', 'Q92', 'Q93', 'Q94', 'Q95', 'Q96', 'Q97', 'Q98', 'Q99', 'Q100']
df.columns.values[1:101]=questions
# print(df.columns.values[1:101])
symbol_number = df.loc[:,df.columns.str.contains('^roll')].fillna(0).agg(lambda x:''.join(x.astype(int).astype(str)), axis=1)
symbol_number.name="Symbol Number"
df

,File name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q98,Q99,Q100,roll.r1,roll.r2,roll.r3,roll.r4,roll.r5,symbol number,Set
0,img08252023_0007-505,C,C,C,C,A,D,B,D,D,...,A,C,A,0.0,0.0,3.0,2.0,3.0,323.0,A
1,img08252023_0007-506,C,B,D,A,A,D,B,D,D,...,B,D,D,0.0,0.0,3.0,4.0,6.0,346.0,A
2,img08252023_0007-503,C,B,D,A,A,D,B,NaN,D,...,B,C,A,0.0,0.0,3.0,4.0,3.0,343.0,A
3,img08252023_0007-504,C,A,C,C,A,D,B,D,D,...,A,C,C,0.0,0.0,3.0,2.0,2.0,322.0,A
4,img08252023_0007-501,C,C,C,A,D,D,NaN,D,D,...,A,D,B,0.0,0.0,3.0,2.0,5.0,325.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,img08252023_0007-362,NaN,A,C,C,C,D,NaN,D,D,...,A,C,A,0.0,0.0,1.0,0.0,7.0,107.0,A
561,img08252023_0007-483,C,A,C,C,A,D,D,C,D,...,B,NaN,D,0.0,0.0,4.0,3.0,8.0,438.0,A
562,img08252023_0007-480,B,A,D,A,D,C,B,D,D,...,A,C,B,0.0,0.0,4.0,2.0,9.0,429.0,A
563,img08252023_0007-360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,...,NaN,NaN,NaN,0.0,0.0,1.0,0.0,1.0,101.0,A


In [13]:
df_with_sn = df.loc[:,~df.columns.str.contains('^roll')].join(symbol_number)
df_with_sn=df_with_sn.set_index("Symbol Number")
df_with_sn.to_csv("Results.csv")
df_with_sn=df_with_sn.drop(columns=["symbol number"])
# df_with_sn=df_with_sn_1.iloc[:,1:]
df_with_sn

,File name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q92,Q93,Q94,Q95,Q96,Q97,Q98,Q99,Q100,Set
Symbol Number,,,,,,,,,,,,,,,,,,,,,
00323,img08252023_0007-505,C,C,C,C,A,D,B,D,D,...,C,B,C,C,B,A,A,C,A,A
00346,img08252023_0007-506,C,B,D,A,A,D,B,D,D,...,B,C,C,C,C,D,B,D,D,A
00343,img08252023_0007-503,C,B,D,A,A,D,B,NaN,D,...,NaN,B,B,B,A,C,B,C,A,A
00322,img08252023_0007-504,C,A,C,C,A,D,B,D,D,...,C,B,D,B,B,A,A,C,C,A
00325,img08252023_0007-501,C,C,C,A,D,D,NaN,D,D,...,C,NaN,C,NaN,C,B,A,D,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00107,img08252023_0007-362,NaN,A,C,C,C,D,NaN,D,D,...,NaN,NaN,NaN,NaN,A,NaN,A,C,A,A
00438,img08252023_0007-483,C,A,C,C,A,D,D,C,D,...,B,D,C,B,NaN,A,B,NaN,D,A
00429,img08252023_0007-480,B,A,D,A,D,C,B,D,D,...,D,B,B,A,A,A,A,C,B,A


In [33]:
df_with_sn_set_a = df_with_sn[df_with_sn["Set"]=="A"].iloc[:,1:-1]
df_with_sn_set_a_group_a = df_with_sn_set_a.iloc[:,0:60]
df_with_sn_set_a_group_b = df_with_sn_set_a.iloc[:,60:]
# print(df_with_sn_set_a_group_a)
df_with_sn_set_a_group_a_nan = df_with_sn_set_a_group_a.isna().sum(axis=1)
df_with_sn_set_a_group_b_nan = df_with_sn_set_a_group_b.isna().sum(axis=1)
df_set_a_correct = df_with_sn_set_a.eq(series_set_a)# Samasya yo line ma chha,
# print(df_set_a_correct)
df_with_sn_set_a
# df_set_a_score = df_set_a_correct.iloc[:,0:60].sum(axis=1)+2*df_set_a_correct.iloc[:,60:].sum(axis=1)
# df_set_a_score =df_set_a_score-(140-(df_set_a_score+df_with_sn_set_a_group_a_nan+2*df_with_sn_set_a_group_b_nan))*0.1
# df_with_sn_set_a.index.is_unique
# df_with_sn_set_a[df_with_sn_set_a.index.duplicated()]



,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q91,Q92,Q93,Q94,Q95,Q96,Q97,Q98,Q99,Q100
Symbol Number,,,,,,,,,,,,,,,,,,,,,
00323,C,C,C,C,A,D,B,D,D,C,...,A,C,B,C,C,B,A,A,C,A
00346,C,B,D,A,A,D,B,D,D,C,...,D,B,C,C,C,C,D,B,D,D
00343,C,B,D,A,A,D,B,NaN,D,A,...,A,NaN,B,B,B,A,C,B,C,A
00322,C,A,C,C,A,D,B,D,D,C,...,A,C,B,D,B,B,A,A,C,C
00325,C,C,C,A,D,D,NaN,D,D,B,...,NaN,C,NaN,C,NaN,C,B,A,D,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00107,NaN,A,C,C,C,D,NaN,D,D,A,...,A,NaN,NaN,NaN,NaN,A,NaN,A,C,A
00438,C,A,C,C,A,D,D,C,D,C,...,A,B,D,C,B,NaN,A,B,NaN,D
00429,B,A,D,A,D,C,B,D,D,C,...,C,D,B,B,A,A,A,A,C,B


In [18]:
df_with_sn_set_b = df_with_sn[df_with_sn["Set"]=="B"].iloc[:,1:-1]
df_with_sn_set_b_group_a = df_with_sn_set_b.iloc[:,0:60]
df_with_sn_set_b_group_b = df_with_sn_set_b.iloc[:,60:]
df_with_sn_set_b_group_a_nan = df_with_sn_set_b_group_a.isna().sum(axis=1)
df_with_sn_set_b_group_b_nan = df_with_sn_set_b_group_b.isna().sum(axis=1)
df_set_b_correct = df_with_sn_set_b.eq(series_set_b)
df_set_b_score = df_set_b_correct.iloc[:,0:60].sum(axis=1)+2*df_set_b_correct.iloc[:,60:].sum(axis=1)
df_set_b_score =df_set_b_score-(140-(df_set_b_score+df_with_sn_set_b_group_a_nan+2*df_with_sn_set_b_group_b_nan))*0.1
df_set_b_score

Series([], dtype: float64)

In [ ]:
df_set_a_correct

,File name,Q1,Q10,Q100,Q11,Q12,Q13,Q14,Q15,Q16,...,c4.Question097,c4.Question098,c4.Question099,c4.Question100,roll.r1,roll.r2,roll.r3,roll.r4,roll.r5,symbol number
Symbol Number,,,,,,,,,,,,,,,,,,,,,
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
df_set_a_score.to_csv("Set A Score.csv")
df_set_b_score.to_csv("Set B Score.csv")